In [21]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import pandas as pd
import math
from tqdm import tqdm

Dalam potongan kode yang diberikan, terdapat beberapa library yang diimpor dan memiliki peran yang berbeda. Pertama, library os digunakan untuk berinteraksi dengan sistem operasi. Fungsionalitas dari os dapat digunakan untuk melakukan operasi terkait file dan direktori, seperti mengakses file dan folder, mendapatkan daftar file dalam sebuah direktori, dan lain sebagainya.

Kemudian, library cv2 atau OpenCV digunakan untuk tugas-tugas pengolahan citra dan video. Dengan menggunakan cv2, Anda dapat membaca, menulis, dan memanipulasi gambar dan video. OpenCV juga menyediakan berbagai fungsi untuk deteksi objek, pengenalan wajah, ekstraksi fitur, serta analisis dan pemrosesan citra secara umum.

Selanjutnya, library numpy digunakan untuk melakukan operasi numerik yang efisien pada array multidimensi. Numpy menyediakan fungsi-fungsi yang kuat dan efisien untuk manipulasi data numerik, termasuk operasi matematika, indexing, dan pemrosesan array.

Library tqdm digunakan untuk membuat bar progres yang interaktif saat melakukan iterasi dalam loop. Dalam konteks potongan kode tersebut, tqdm mungkin digunakan untuk memberikan indikator visual tentang kemajuan iterasi pada loop tertentu.

Terakhir, library pandas menyediakan struktur data dan fungsi untuk analisis data yang efisien. Dengan menggunakan pandas, Anda dapat membaca, menulis, dan memanipulasi data dalam berbagai format seperti CSV, Excel, SQL, dan lain sebagainya. pandas juga menyediakan fitur-fitur untuk indexing, filtering, penggabungan data, dan komputasi statistik yang kuat.

In [22]:
def normalize_resize_images(folder):
    normalized_images = []
    
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        
        if os.path.isdir(label_path):
            print(f"Memproses gambar pada folder: {label}")
            
            for filename in os.listdir(label_path):
                file_path = os.path.join(label_path, filename)
                img = cv2.imread(file_path)
                
                if img is not None:
                    normalized_image = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
                    
                    resized_image = cv2.resize(normalized_image, (256, 256))
            
                    normalized_images.append(resized_image)
                    
                else:
                    print(f"Gagal membaca gambar: {file_path}")
                    
        else:
            print(f"Label '{label}' tidak dalam direktori")
    print('===========================================')        
    print("Normalisasi dan resize selesai^^")
    print('\n')
    return normalized_images

Fungsi normalize_resize_images(folder) digunakan untuk melakukan normalisasi dan perubahan ukuran (resize) pada gambar-gambar yang berada dalam folder yang ditentukan. Pertama, fungsi melakukan iterasi melalui setiap label (subfolder) yang ada dalam folder tersebut. Setiap gambar dalam setiap label di-load menggunakan cv2.imread() untuk membaca gambar sebagai array piksel. Jika gambar berhasil dibaca, gambar tersebut di-normalisasi menggunakan cv2.normalize(), dengan rentang nilai 0-255, sehingga nilai pikselnya menjadi lebih terdistribusi secara normal. Setelah itu, gambar diubah ukurannya menjadi 256x256 piksel menggunakan cv2.resize(). Gambar yang telah dinormalisasi dan diubah ukurannya ditambahkan ke dalam list normalized_images. Jika ada kesalahan saat membaca gambar, pesan kesalahan akan ditampilkan. Setelah selesai memproses semua gambar dalam folder, fungsi mengembalikan list normalized_images yang berisi gambar-gambar yang telah dinormalisasi dan diubah ukurannya.

In [23]:
def derajat0(img):
    max_val = np.max(img)
    img_tmp = np.zeros([max_val + 1, max_val + 1])
    
    for i in range(len(img)):
        for j in range(len(img[i]) - 1):
            img_tmp[img[i, j], img[i, j + 1]] += 1
    
    data = img_tmp + np.transpose(img_tmp)
    tmp = np.sum(data)
    
    for i in range(len(data)):
        for j in range(len(data)):
            data[i, j] /= tmp
    
    return data

def derajat45(img):
    max_val = np.max(img)
    img_tmp = np.zeros([max_val + 1, max_val + 1])
    
    for i in range(len(img) - 1):
        for j in range(len(img[i]) - 1):
            img_tmp[img[i + 1, j], img[i, j + 1]] += 1
    
    data = img_tmp + np.transpose(img_tmp)
    tmp = np.sum(data)
    
    for i in range(len(data)):
        for j in range(len(data)):
            data[i, j] /= tmp
    
    return data

def derajat90(img):
    max_val = np.max(img)
    img_tmp = np.zeros([max_val + 1, max_val + 1])
    
    for i in range(len(img) - 1):
        for j in range(len(img[i])):
            img_tmp[img[i + 1, j], img[i, j]] += 1
    
    data = img_tmp + np.transpose(img_tmp)
    tmp = np.sum(data)
    
    for i in range(len(data)):
        for j in range(len(data)):
            data[i, j] /= tmp
    
    return data

def derajat135(img):
    max_val = np.max(img)
    img_tmp = np.zeros([max_val + 1, max_val + 1])
    
    for i in range(len(img) - 1):
        for j in range(len(img[i]) - 1):
            img_tmp[img[i, j], img[i + 1, j + 1]] += 1
    
    data = img_tmp + np.transpose(img_tmp)
    tmp = np.sum(data)
    
    for i in range(len(data)):
        for j in range(len(data)):
            data[i, j] /= tmp
    
    return data

def extract_features(normalized_images):
    features = []
    
    for img in tqdm(normalized_images, desc='Ekstraksi fitur GLCM'):
        dat = []
        dat.append(derajat0(img))
        dat.append(derajat45(img))
        dat.append(derajat90(img))
        dat.append(derajat135(img))
        features.append(dat)
        
    return features

# Mendefinisikan path folder dataset
folder_path = "fast-food"

# Mengambil gambar, normalisasi, dan resize
images = normalize_resize_images(folder_path)

# Mengambil subset gambar sesuai dengan batasan
batas_gambar = 25
subset_images = images[:batas_gambar]

# Ekstraksi fitur GLCM
features = extract_features(subset_images)

Memproses gambar pada folder: Baked Potato
Memproses gambar pada folder: Burger
Memproses gambar pada folder: Crispy Chicken
Memproses gambar pada folder: Donut
Memproses gambar pada folder: Fries
Memproses gambar pada folder: Hot Dog
Memproses gambar pada folder: Pizza
Memproses gambar pada folder: Sandwich
Memproses gambar pada folder: Taco
Memproses gambar pada folder: Taquito
Normalisasi dan resize selesai^^




Ekstraksi fitur GLCM: 100%|██████████| 25/25 [00:43<00:00,  1.74s/it]


Fungsi derajat0(img) pada potongan kode tersebut berfungsi untuk menghasilkan matriks GLCM (Gray Level Co-occurrence Matrix) pada sudut 0 derajat (horisontal) dari sebuah gambar img. Langkah-langkah yang dilakukan oleh fungsi ini adalah sebagai berikut: Pertama, fungsi mencari nilai maksimum dalam gambar menggunakan np.max(img). Kemudian, matriks img_tmp dibuat sebagai matriks nol dengan ukuran (max_val + 1, max_val + 1). Selanjutnya, dilakukan iterasi melalui piksel-piksel gambar menggunakan nested loop. Pada setiap iterasi, kemunculan pasangan piksel pada sudut 0 derajat (horisontal) dihitung dengan menambahkan 1 ke elemen matriks img_tmp pada indeks yang sesuai. Setelah selesai iterasi, matriks img_tmp dijumlahkan dengan transpose-nya menggunakan np.transpose(img_tmp) untuk menghasilkan matriks GLCM simetris. Selanjutnya, dilakukan normalisasi pada setiap elemen matriks GLCM dengan membaginya dengan jumlah total kemunculan piksel yang dihitung menggunakan np.sum(data). Hasilnya, matriks GLCM yang telah dinormalisasi dikembalikan sebagai output fungsi.

Fungsi derajat45(img) pada potongan kode tersebut bertujuan untuk menghasilkan matriks GLCM (Gray Level Co-occurrence Matrix) pada sudut 45 derajat dari sebuah gambar img. Langkah-langkah yang dilakukan oleh fungsi ini meliputi: pertama, menghitung nilai maksimum dalam gambar menggunakan np.max(img) dan menyimpannya dalam variabel max_val. Selanjutnya, matriks img_tmp dengan ukuran (max_val + 1, max_val + 1) dibuat sebagai matriks nol. Kemudian, dilakukan iterasi melalui piksel-piksel gambar menggunakan nested loop. Pada setiap iterasi, kemunculan pasangan piksel pada sudut 45 derajat dihitung dengan menambahkan 1 ke elemen matriks img_tmp pada indeks yang sesuai. Setelah selesai iterasi, matriks img_tmp dijumlahkan dengan transpose-nya menggunakan np.transpose(img_tmp) untuk menghasilkan matriks GLCM yang simetris. Selanjutnya, dilakukan normalisasi pada setiap elemen matriks GLCM dengan membaginya dengan jumlah total kemunculan piksel yang dihitung menggunakan np.sum(data). Hasilnya, matriks GLCM yang telah dinormalisasi dikembalikan sebagai output fungsi.

Fungsi derajat90(img) pada potongan kode tersebut berfungsi untuk menghasilkan matriks GLCM (Gray Level Co-occurrence Matrix) pada sudut 90 derajat dari sebuah gambar img. Langkah-langkah yang dilakukan oleh fungsi ini adalah sebagai berikut: Pertama, variabel max_val diinisialisasi dengan nilai maksimum dalam gambar menggunakan np.max(img). Variabel ini digunakan untuk menentukan ukuran matriks GLCM. Selanjutnya, matriks img_tmp dibuat sebagai matriks nol dengan ukuran (max_val + 1, max_val + 1). Dilakukan iterasi melalui piksel-piksel gambar menggunakan nested loop. Pada setiap iterasi, kemunculan pasangan piksel pada sudut 90 derajat dihitung dengan menambahkan 1 ke elemen matriks img_tmp pada indeks yang sesuai. Setelah selesai iterasi, matriks img_tmp dijumlahkan dengan transpose-nya menggunakan np.transpose(img_tmp) untuk menghasilkan matriks GLCM simetris. Selanjutnya, dilakukan normalisasi pada setiap elemen matriks GLCM dengan membaginya dengan jumlah total kemunculan piksel yang dihitung menggunakan np.sum(data). Hasilnya, matriks GLCM yang telah dinormalisasi dikembalikan sebagai output fungsi.


Fungsi derajat135(img) pada potongan kode tersebut berfungsi untuk menghasilkan matriks GLCM (Gray Level Co-occurrence Matrix) pada sudut 135 derajat dari sebuah gambar img. Langkah-langkah yang dilakukan oleh fungsi ini meliputi: pertama, nilai maksimum dalam gambar dihitung menggunakan np.max(img) dan disimpan dalam variabel max_val. Selanjutnya, matriks img_tmp dengan ukuran (max_val + 1, max_val + 1) dibuat sebagai matriks nol. Dilakukan iterasi melalui piksel-piksel gambar menggunakan nested loop. Pada setiap iterasi, kemunculan pasangan piksel pada sudut 135 derajat dihitung dengan menambahkan 1 ke elemen matriks img_tmp pada indeks yang sesuai. Setelah selesai iterasi, matriks img_tmp dijumlahkan dengan transpose-nya menggunakan np.transpose(img_tmp) untuk menghasilkan matriks GLCM yang simetris. Selanjutnya, dilakukan normalisasi pada setiap elemen matriks GLCM dengan membaginya dengan jumlah total kemunculan piksel yang dihitung menggunakan np.sum(data). Hasilnya, matriks GLCM yang telah dinormalisasi dikembalikan sebagai output fungsi.


Fungsi extract_features(normalized_images) pada potongan kode tersebut bertujuan untuk mengekstraksi fitur GLCM (Gray Level Co-occurrence Matrix) dari kumpulan gambar yang telah dinormalisasi. Langkah-langkah yang dilakukan oleh fungsi ini meliputi: pertama, variabel features diinisialisasi sebagai sebuah list kosong yang akan menyimpan hasil ekstraksi fitur. Selanjutnya, dilakukan iterasi melalui setiap gambar yang telah dinormalisasi menggunakan loop for. Pada setiap iterasi, sebuah list dat diinisialisasi sebagai kontainer untuk menyimpan hasil ekstraksi fitur GLCM dari gambar tersebut. Selanjutnya, fungsi-fungsi derajat0(), derajat45(), derajat90(), dan derajat135() dipanggil untuk menghasilkan matriks GLCM pada berbagai sudut dari gambar yang sedang diproses. Matriks-matriks GLCM ini kemudian ditambahkan ke dalam list dat. Setelah selesai iterasi, list dat yang berisi matriks-matriks GLCM dari satu gambar dikumpulkan dalam list features. Pada akhirnya, list features yang berisi hasil ekstraksi fitur GLCM dari seluruh gambar dikembalikan sebagai output fungsi.



In [24]:
def contrast(data):
    contrast = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            contrast += data[i][j] * pow(i - j, 2)
    return contrast

def dissimilarity(data):
    dissimilarity = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            dissimilarity += abs(i - j) * data[i][j]
    return dissimilarity

def homogeneity(data):
    homogeneity = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            homogeneity += data[i][j] / (1 + pow(i - j, 2))
    return homogeneity

def entropy(data):
    entropy = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            if data[i][j] > 0.0:
                entropy += -(data[i][j] * math.log(data[i][j]))
    return entropy

def asm(data):
    asm = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            asm += data[i][j] ** 2
    return asm

def energy(data):
    energy = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            energy += data[i][j] ** 2
    return energy

def correlation(data):
    correlation = 0
    mean_i = 0
    mean_j = 0
    std_i = 0
    std_j = 0
    for i in range(len(data)):
        for j in range(len(data[0])):
            correlation += ((i * j) * data[i][j])
            mean_i += i * data[i][j]
            mean_j += j * data[i][j]
    
    for i in range(len(data)):
        for j in range(len(data[0])):
            std_i += ((i - mean_i) ** 2) * data[i][j]
            std_j += ((j - mean_j) ** 2) * data[i][j]
    
    std_i = math.sqrt(std_i)
    std_j = math.sqrt(std_j)
    
    if std_i != 0 and std_j != 0:
        correlation = (correlation - (mean_i * mean_j)) / (std_i * std_j)
    
    return correlation

data0 = []
angles = ['0', '45', '90', '135']
result = []

batas_gambar = 10000

for j in tqdm(range(min(len(features), batas_gambar)), desc="Ekstraksi"):
    extracted_data = []
    extracted_data.append(folder_path)  # Menggunakan folder_path sebagai informasi folder
    
    for i in features[j]:
        contrast_val = contrast(i)
        extracted_data.append(contrast_val)

        dissimilarity_val = dissimilarity(i)
        extracted_data.append(dissimilarity_val)

        homogeneity_val = homogeneity(i)
        extracted_data.append(homogeneity_val)

        entropy_val = entropy(i)
        extracted_data.append(entropy_val)

        asm_val = asm(i)
        extracted_data.append(asm_val)

        energy_val = energy(i)
        extracted_data.append(energy_val)

        correlation_val = correlation(i)
        extracted_data.append(correlation_val)

    result.append(extracted_data)

headers = ['path']
for angle in angles:
    headers.extend([f'kontras{angle}', f'dissimilarity{angle}', f'homogenitas{angle}', f'entropi{angle}', f'asm{angle}', f'energi{angle}', f'correlation{angle}'])

df = pd.DataFrame(result, columns=headers)
df.head(25)

Ekstraksi: 100%|██████████| 25/25 [00:29<00:00,  1.19s/it]


,path,kontras0,dissimilarity0,homogenitas0,entropi0,asm0,energi0,correlation0,kontras45,dissimilarity45,...,asm90,energi90,correlation90,kontras135,dissimilarity135,homogenitas135,entropi135,asm135,energi135,correlation135
0,fast-food,119.902148,6.495143,0.236617,8.935161,0.000215,0.000215,0.988120,193.817743,8.487594,...,0.000192,0.000192,0.985565,303.977623,10.170952,0.157965,9.353393,0.000135,0.000135,0.969961
1,fast-food,177.113375,8.414019,0.165649,8.993681,0.000221,0.000221,0.966850,245.742587,9.841514,...,0.000355,0.000355,0.983513,238.906506,9.875234,0.139765,9.143984,0.000187,0.000187,0.955231
2,fast-food,373.698639,11.029849,0.195809,9.195381,0.000381,0.000381,0.973053,1216.714388,21.202300,...,0.000213,0.000213,0.932393,1225.271848,21.366223,0.111690,9.794002,0.000173,0.000173,0.911803
3,fast-food,98.977530,6.355463,0.218024,8.891338,0.000311,0.000311,0.991172,393.447676,12.851459,...,0.000179,0.000179,0.972709,367.452519,12.509203,0.116760,9.517985,0.000158,0.000158,0.967247
4,fast-food,135.501241,7.209647,0.217773,8.938966,0.000280,0.000280,0.984051,497.157691,14.181710,...,0.000169,0.000169,0.958306,401.715437,12.580397,0.134203,9.462730,0.000154,0.000154,0.952763
5,fast-food,349.908836,12.098722,0.149176,9.550024,0.000114,0.000114,0.964862,900.785941,20.481819,...,0.000087,0.000087,0.933776,924.176709,20.910524,0.074852,10.067409,0.000060,0.000060,0.907304
6,fast-food,61.596809,3.506947,0.554388,6.293410,0.101214,0.101214,0.992741,141.757668,5.564778,...,0.099739,0.099739,0.988541,146.768250,5.853234,0.467890,6.654032,0.097932,0.097932,0.982688
7,fast-food,158.431213,6.848750,0.280920,8.643985,0.000710,0.000710,0.981241,501.592679,11.985484,...,0.000542,0.000542,0.955699,460.373922,11.149994,0.192234,9.071865,0.000444,0.000444,0.945575
8,fast-food,149.741287,7.590357,0.232113,8.933679,0.000323,0.000323,0.978221,202.326495,8.995723,...,0.000427,0.000427,0.989963,197.117489,8.720786,0.205164,9.068875,0.000269,0.000269,0.971362
9,fast-food,175.700240,7.685661,0.290144,8.646782,0.000993,0.000993,0.978751,390.676463,12.209749,...,0.000741,0.000741,0.974928,306.519974,10.649219,0.204080,9.010153,0.000623,0.000623,0.962947


Fungsi contrast(data) pada potongan kode tersebut digunakan untuk menghitung kontras dari sebuah matriks data. Kontras merupakan salah satu metrik yang digunakan untuk mengukur variasi intensitas dalam suatu gambar. Langkah-langkah yang dilakukan oleh fungsi ini meliputi: pertama, variabel contrast diinisialisasi sebagai nol. Selanjutnya, dilakukan iterasi melalui setiap elemen dalam matriks data menggunakan nested loop. Pada setiap iterasi, kontribusi dari elemen tersebut terhadap kontras dihitung dengan mengalikan nilai elemen dengan kuadrat selisih antara indeks baris dan indeks kolom. Hasil kontribusi dari setiap elemen ditambahkan ke variabel contrast. Setelah selesai iterasi, nilai contrast yang merupakan akumulasi kontribusi dari seluruh elemen dalam matriks dikembalikan sebagai output fungsi. Dengan menggunakan fungsi contrast(data), kita dapat menghitung nilai kontras dari matriks data yang dapat digunakan sebagai fitur dalam analisis citra atau pengolahan gambar.

Fungsi dissimilarity(data) pada potongan kode tersebut digunakan untuk menghitung dissimilarity (ketidakmiripan) dari sebuah matriks data. Dissimilarity merupakan salah satu metrik yang digunakan untuk mengukur perbedaan intensitas antara pasangan piksel dalam suatu gambar. Langkah-langkah yang dilakukan oleh fungsi ini meliputi: pertama, variabel dissimilarity diinisialisasi sebagai nol. Selanjutnya, dilakukan iterasi melalui setiap elemen dalam matriks data menggunakan nested loop. Pada setiap iterasi, kontribusi dari elemen tersebut terhadap dissimilarity dihitung dengan mengalikan selisih absolut antara indeks baris dan indeks kolom dengan nilai elemen. Hasil kontribusi dari setiap elemen ditambahkan ke variabel dissimilarity. Setelah selesai iterasi, nilai dissimilarity yang merupakan akumulasi kontribusi dari seluruh elemen dalam matriks dikembalikan sebagai output fungsi. Dengan menggunakan fungsi dissimilarity(data), kita dapat menghitung nilai dissimilarity dari matriks data yang dapat digunakan sebagai fitur dalam analisis citra atau pengolahan gambar.

Fungsi homogeneity(data) pada potongan kode tersebut digunakan untuk menghitung homogeneity (homogenitas) dari sebuah matriks data. Homogeneity merupakan salah satu metrik yang digunakan untuk mengukur tingkat keseragaman atau keterhubungan intensitas dalam suatu gambar. Langkah-langkah yang dilakukan oleh fungsi ini meliputi: pertama, variabel homogeneity diinisialisasi sebagai nol. Selanjutnya, dilakukan iterasi melalui setiap elemen dalam matriks data menggunakan nested loop. Pada setiap iterasi, kontribusi dari elemen tersebut terhadap homogeneity dihitung dengan membagi nilai elemen dengan penjumlahan 1 dan kuadrat selisih antara indeks baris dan indeks kolom. Hasil kontribusi dari setiap elemen ditambahkan ke variabel homogeneity. Setelah selesai iterasi, nilai homogeneity yang merupakan akumulasi kontribusi dari seluruh elemen dalam matriks dikembalikan sebagai output fungsi. Dengan menggunakan fungsi homogeneity(data), kita dapat menghitung nilai homogeneity dari matriks data yang dapat digunakan sebagai fitur dalam analisis citra atau pengolahan gambar.

Fungsi entropy(data) pada potongan kode tersebut digunakan untuk menghitung entropy (entropi) dari sebuah matriks data. Entropy merupakan salah satu metrik yang digunakan untuk mengukur tingkat ketidakpastian atau kompleksitas distribusi probabilitas dari suatu variabel acak. Langkah-langkah yang dilakukan oleh fungsi ini adalah sebagai berikut: pertama, variabel entropy diinisialisasi sebagai nol. Selanjutnya, dilakukan iterasi melalui setiap elemen dalam matriks data menggunakan nested loop. Pada setiap iterasi, jika nilai elemen tersebut lebih besar dari nol, kontribusinya terhadap entropy dihitung dengan mengalikan nilai elemen dengan logaritma basis 10 dari nilai elemen tersebut yang kemudian dikalikan dengan minus satu. Hasil kontribusi dari setiap elemen yang memenuhi kondisi tersebut ditambahkan ke variabel entropy. Setelah selesai iterasi, nilai entropy yang merupakan akumulasi kontribusi dari seluruh elemen dalam matriks dikembalikan sebagai output fungsi. Dengan menggunakan fungsi entropy(data), kita dapat menghitung nilai entropy dari matriks data yang dapat digunakan sebagai fitur dalam analisis citra atau pengolahan gambar.

Fungsi asm(data) pada potongan kode tersebut digunakan untuk menghitung Angular Second Moment (ASM) dari sebuah matriks data. ASM merupakan salah satu metrik yang digunakan untuk mengukur tingkat keteraturan atau kehalusan distribusi intensitas dalam suatu gambar. Langkah-langkah yang dilakukan oleh fungsi ini meliputi: pertama, variabel asm diinisialisasi sebagai nol. Selanjutnya, dilakukan iterasi melalui setiap elemen dalam matriks data menggunakan nested loop. Pada setiap iterasi, kontribusi dari elemen tersebut terhadap ASM dihitung dengan mengkuadratkan nilai elemen dan menambahkannya ke variabel asm. Setelah selesai iterasi, nilai asm yang merupakan akumulasi kontribusi kuadrat dari seluruh elemen dalam matriks dikembalikan sebagai output fungsi. Dengan menggunakan fungsi asm(data), kita dapat menghitung nilai ASM dari matriks data yang dapat digunakan sebagai fitur dalam analisis citra atau pengolahan gambar.

Fungsi energy(data) pada potongan kode tersebut digunakan untuk menghitung energy (energi) dari sebuah matriks data. Energy merupakan metrik yang mengukur jumlah energi atau kekuatan distribusi intensitas dalam suatu gambar. Langkah-langkah yang dilakukan oleh fungsi ini adalah sebagai berikut: pertama, variabel energy diinisialisasi sebagai nol. Selanjutnya, dilakukan iterasi melalui setiap elemen dalam matriks data menggunakan nested loop. Pada setiap iterasi, kontribusi dari elemen tersebut terhadap energy dihitung dengan mengkuadratkan nilai elemen dan menambahkannya ke variabel energy. Setelah selesai iterasi, nilai energy yang merupakan akumulasi kontribusi kuadrat dari seluruh elemen dalam matriks dikembalikan sebagai output fungsi. Dengan menggunakan fungsi energy(data), kita dapat menghitung nilai energy dari matriks data yang dapat digunakan sebagai fitur dalam analisis citra atau pengolahan gambar.

Fungsi correlation(data) pada potongan kode tersebut digunakan untuk menghitung korelasi (correlation) dari sebuah matriks data. Korelasi mengukur tingkat hubungan linier antara dua variabel, dalam hal ini intensitas piksel dalam gambar. Fungsi ini melakukan iterasi melalui setiap elemen dalam matriks data dan menghitung kontribusi terhadap korelasi dengan mengalikan koordinat (i, j) dengan nilai elemen data[i][j]. Selain itu, fungsi juga menghitung mean dari variabel i dan j, serta menghitung standard deviation dari variabel i dan j. Setelah itu, dilakukan pengecekan untuk menghindari pembagian dengan nol saat perhitungan korelasi. Jika standard deviation tidak nol, maka dilakukan perhitungan korelasi dengan mengurangi perkalian mean dari i dan j dari korelasi dan membaginya dengan perkalian standard deviation dari i dan j. Hasil korelasi kemudian dikembalikan sebagai output dari fungsi.

Potongan kode tersebut merupakan bagian dari proses ekstraksi fitur dari matriks GLCM (Gray Level Co-occurrence Matrix). Pada loop for j in tqdm(range(min(len(features), batas_gambar)), desc="Ekstraksi"), dilakukan iterasi sebanyak batas_gambar atau jumlah fitur yang ingin diekstraksi. Pada setiap iterasi, dilakukan ekstraksi fitur seperti kontras, dissimilarity, homogeneity, entropy, asm, energy, dan correlation dari matriks GLCM yang disimpan dalam variabel features. Hasil ekstraksi fitur kemudian disimpan dalam variabel extracted_data dan ditambahkan dengan informasi folder dari folder_path. Setelah itu, hasil ekstraksi fitur dari semua gambar disimpan dalam list result. Selanjutnya, dibentuk dataframe menggunakan pd.DataFrame dari library Pandas dengan menggunakan result sebagai data dan headers sebagai kolom. Dataframe tersebut akan berisi hasil ekstraksi fitur yang dapat digunakan untuk analisis lebih lanjut.

In [25]:
# Mengambil subset data frame tanpa kolom 'path'
subset_df = df.drop('path', axis=1)

# Menghitung matriks korelasi
n_features = len(subset_df.columns)
corr_matrix = np.zeros((n_features, n_features))

for i in range(n_features):
    for j in range(i+1, n_features):
        feature1 = subset_df.columns[i]
        feature2 = subset_df.columns[j]
        data1 = subset_df[feature1].values
        data2 = subset_df[feature2].values

        valid_indices = np.isfinite(data1) & np.isfinite(data2)
        if np.sum(valid_indices) > 0:
            corr = np.corrcoef(data1[valid_indices], data2[valid_indices])[0, 1]
            corr_matrix[i, j] = corr
            corr_matrix[j, i] = corr

# Mengambil 5 korelasi tertinggi
top_correlations = []
for i in range(n_features):
    for j in range(i+1, n_features):
        feature1 = subset_df.columns[i]
        feature2 = subset_df.columns[j]
        correlation = corr_matrix[i, j]
        top_correlations.append(((feature1, feature2), correlation))

top_correlations.sort(key=lambda x: x[1], reverse=True)
top_correlations = top_correlations[:5]

print("5 Korelasi Tertinggi:")
for (feature1, feature2), correlation in top_correlations:
    print(f"{feature1} - {feature2}: {correlation}")

5 Korelasi Tertinggi:
asm0 - energi0: 1.0
asm45 - energi45: 1.0
asm90 - energi90: 1.0
asm135 - energi135: 1.0
asm45 - asm135: 0.9999998404793254



Potongan kode tersebut digunakan untuk menghitung matriks korelasi antara fitur-fitur yang diekstraksi. Pertama, kolom 'path' dihapus dari dataframe df menggunakan df.drop('path', axis=1), sehingga diperoleh dataframe subset_df yang hanya berisi fitur-fitur yang akan digunakan. Selanjutnya, sebuah matriks kosong corr_matrix dibentuk dengan ukuran sesuai dengan jumlah fitur. Melalui iterasi, korelasi antara setiap pasangan fitur dihitung menggunakan np.corrcoef dan hasilnya disimpan dalam matriks corr_matrix. Selanjutnya, dilakukan pencarian 5 korelasi tertinggi dengan menyimpan pasangan fitur dan nilai korelasinya dalam daftar top_correlations, yang kemudian diurutkan berdasarkan nilai korelasi secara menurun. Hasil akhirnya adalah menampilkan 5 korelasi tertinggi yang ditemukan beserta fitur-fiturnya.